In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import urllib
import sys
from matplotlib import pyplot as plt
#%matplotlib inline
import altair as alt
import pandas as pd

In [2]:
import urllib.request


url_template = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-{:02d}.parquet"
year = "2022"

all_data = []

for month in range(1, 3):
    url = url_template.format(month)
    file_name = f"yellow_trips_{year}_{month:02d}.parquet"
    print(f"Downloading {file_name}...")
    urllib.request.urlretrieve(url, file_name)
    data = pd.read_parquet(file_name)
    all_data.append(data)

# Concatenate all data into a single DataFrame
data = pd.concat(all_data, ignore_index=True)


In [3]:
data['trip_duration'] = (data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime'])
data['trip_duration'] = data['trip_duration'].dt.total_seconds()/60
data['speed'] = data['trip_distance']/(data['trip_duration']/60)
data['year'] = data['tpep_dropoff_datetime'].dt.year
data['month'] = data['tpep_dropoff_datetime'].dt.month
data['day'] = data['tpep_dropoff_datetime'].dt.day
data['DOW'] = data['tpep_dropoff_datetime'].dt.dayofweek
data['hour'] = data['tpep_dropoff_datetime'].dt.hour
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,total_amount,congestion_surcharge,airport_fee,trip_duration,speed,year,month,day,DOW,hour
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,...,21.95,2.5,0.0,17.816667,12.797007,2022,1,1,5,0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,...,13.30,0.0,0.0,8.400000,15.000000,2022,1,1,5,0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,...,10.56,0.0,0.0,8.966667,6.490706,2022,1,1,5,1
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,...,11.80,2.5,0.0,10.033333,6.518272,2022,1,1,5,0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,...,30.30,2.5,0.0,37.533333,6.873890,2022,1,1,5,1


In [4]:
df = data[['PULocationID','DOLocationID','trip_duration','day','year','month','DOW','hour']]

In [5]:
taxi = gpd.read_file('https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=Shapefile')
taxi.head()

,borough,location_i,objectid,shape_area,shape_leng,zone,geometry
0,EWR,1.0,1.0,0.000782,0.116357,Newark Airport,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
1,Queens,2.0,2.0,0.004866,0.433470,Jamaica Bay,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,Bronx,3.0,3.0,0.000314,0.084341,Allerton/Pelham Gardens,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,Manhattan,4.0,4.0,0.000112,0.043567,Alphabet City,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,Staten Island,5.0,5.0,0.000498,0.092146,Arden Heights,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


In [6]:
#data clean
df = df[(df.PULocationID.isin(taxi.location_i)) & (df.DOLocationID.isin(taxi.location_i))]
df = df.loc[df['trip_duration'] > 0]

In [7]:
df.head()

,PULocationID,DOLocationID,trip_duration,day,year,month,DOW,hour
0,142,236,17.816667,1,2022,1,5,0
1,236,42,8.400000,1,2022,1,5,0
2,166,166,8.966667,1,2022,1,5,1
3,114,68,10.033333,1,2022,1,5,0
4,68,163,37.533333,1,2022,1,5,1


In [8]:
PD = df.groupby(by=['PULocationID','DOLocationID']).agg({'trip_duration':'mean','day':'count'})\
        .reset_index().reset_index(drop=True).rename(columns={'day': 'count'})
PD['trip_duration'] = PD['trip_duration'].astype(int)

In [9]:
Time = df.groupby(by=['PULocationID','DOLocationID','hour']).agg({'trip_duration':'mean','day':'count'})\
        .reset_index().reset_index(drop=True).rename(columns={'day': 'count'})

In [10]:
Time_byDOW = df.groupby(by=['PULocationID','DOLocationID','hour','DOW'])\
        .agg({'trip_duration':'mean','day':'count'}).reset_index()\
        .reset_index(drop=True).rename(columns={'day': 'count'})

In [11]:
#@title Spatial distribution of the taxi activity

#get the number of trips by taxi zone of origin
tripsByZone = df.groupby(by='PULocationID', as_index=False).count()[['PULocationID', 'DOLocationID']]
tripsByZone.rename(columns={'DOLocationID':'N_trips'}, inplace=True)
tripsByZone.head()

,PULocationID,N_trips
0,1,636
1,2,5
2,3,71
3,4,6754
4,5,71


In [12]:
#@title plot the number of trips per taxi zone on a NYC map
tripsViz = taxi.merge(tripsByZone, left_on='location_i', right_on='PULocationID')[['zone','geometry','PULocationID','N_trips']]#.rename(columns={'PULocationID': 'location_ID'})
tripsViz.head()

,zone,geometry,PULocationID,N_trips
0,Newark Airport,"POLYGON ((-74.18445 40.69500, -74.18449 40.695...",1,636
1,Jamaica Bay,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",2,5
2,Allerton/Pelham Gardens,"POLYGON ((-73.84793 40.87134, -73.84725 40.870...",3,71
3,Alphabet City,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",4,6754
4,Arden Heights,"POLYGON ((-74.17422 40.56257, -74.17349 40.562...",5,71


In [13]:
import json

tripsViz_geojson = json.loads(tripsViz.to_json())


In [14]:
import altair as alt
import numpy as np
import pandas as pd

alt.data_transformers.disable_max_rows()

color_scale = alt.Scale(scheme='blueorange', domain=np.percentile(tripsViz['N_trips'], [10, 20, 30, 40, 50, 60, 70, 80, 90]))

pu_location_selection = alt.selection_single(fields=['PULocationID'], on='click', empty='all', name='PULocation')
do_location_selection = alt.selection_single(fields=['DOLocationID'], on='click', empty='all', name='DOLocation')

trips_map = alt.Chart(alt.Data(values=tripsViz_geojson['features']))\
    .mark_geoshape(stroke='white', strokeWidth=1)\
    .encode(
        tooltip=['properties.PULocationID:O', 'properties.N_trips:Q'],
        color=alt.condition(pu_location_selection | do_location_selection, 'properties.N_trips:Q', alt.value('lightgray'), scale=color_scale, legend=alt.Legend(title='Number of Trips (Quantiles)')),
        opacity=alt.value(1)
    )\
    .project('albersUsa')\
    .transform_calculate(PULocationID='datum.properties.PULocationID', DOLocationID='datum.properties.PULocationID')\
    .add_selection(pu_location_selection)\
    .add_selection(do_location_selection)

bars = alt.Chart(PD)\
    .mark_circle()\
    .encode(
        x="PULocationID:O",
        y="DOLocationID:O",
        size="trip_duration:Q",
        color=alt.Color("trip_duration:Q", scale=alt.Scale(scheme="viridis")),
        opacity=alt.condition(pu_location_selection, alt.value(1), alt.value(0))
    )\
    .transform_filter(pu_location_selection)

# Line chart with selection
line_chart = alt.Chart(Time)\
    .mark_line()\
    .encode(
        x=alt.X("hour:O", axis=alt.Axis(title="Hour")),
        y=alt.Y("trip_duration:Q", axis=alt.Axis(title="Trip Duration")),
    )\
    .transform_filter(pu_location_selection)\
    .transform_filter(do_location_selection)

# Create an empty dataframe for the empty line chart
empty_df = pd.DataFrame({'hour': [], 'trip_duration': []})

# Create an empty line chart to display when no location is selected
empty_line_chart = alt.Chart(empty_df).mark_line(color='white').encode(x="hour:O", y="trip_duration:Q")

# Display the empty line chart when no location is selected, otherwise display the line chart with selected location
combined_line_chart = alt.layer(empty_line_chart, line_chart).transform_filter(pu_location_selection | do_location_selection)

combined_map_bars = trips_map.properties(width=600, height=800, title='New York City Taxi Trips by Zone') | bars.properties(width=800, height=800, title='Trip Time')
chart = combined_map_bars & combined_line_chart.properties(width=800, height=400, title='Trip Duration by Hour')

#chart


In [15]:
pu_location_selection = alt.selection_single(fields=['PULocationID'], on='click', empty='all', name='PULocation')
do_location_selection = alt.selection_single(fields=['DOLocationID'], on='mouseover', empty='all', name='DOLocation')

trips_map = alt.Chart(alt.Data(values=tripsViz_geojson['features']))\
    .mark_geoshape(stroke='white', strokeWidth=1)\
    .encode(
        tooltip=['properties.PULocationID:O', 'properties.N_trips:Q'],
        color=alt.condition(pu_location_selection | do_location_selection, 'properties.N_trips:Q', alt.value('lightgray'), scale=color_scale, legend=alt.Legend(title='Number of Trips (Quantiles)')),
        opacity=alt.value(1)
    )\
    .project('albersUsa')\
    .transform_calculate(PULocationID='datum.properties.PULocationID', DOLocationID='datum.properties.PULocationID')\
    .add_selection(pu_location_selection)\
    .add_selection(do_location_selection)

bars = alt.Chart(PD)\
    .mark_circle()\
    .encode(
        x="PULocationID:O",
        y="DOLocationID:O",
        size="trip_duration:Q",
        color=alt.Color("trip_duration:Q", scale=alt.Scale(scheme="viridis")),
        opacity=alt.condition(
            (pu_location_selection & do_location_selection) |
            (pu_location_selection & ~do_location_selection),
            alt.value(1), 
            alt.value(0)
        )
    )\
    .transform_filter(pu_location_selection)\
    .transform_filter(
        do_location_selection |
        (pu_location_selection & ~do_location_selection)
    )



selected_zone_line_chart = alt.Chart(Time)\
    .mark_line(color='orange')\
    .encode(
        x=alt.X("hour:O", axis=alt.Axis(title="Hour")),
        y=alt.Y("trip_duration:Q", axis=alt.Axis(title="Trip Duration")),
    )\
    .transform_filter(pu_location_selection)\
    .transform_filter(do_location_selection)

combined_map_bars = trips_map.properties(width=600, height=800, title='New York City Taxi Trips by Zone') | bars.properties(width=800, height=800, title='Trip Time')
chart = combined_map_bars & combined_line_chart.properties(width=1400, height=400, title='Trip Duration by Hour')

#chart


In [16]:
Time_byDOW

,PULocationID,DOLocationID,hour,DOW,trip_duration,count
0,1,1,0,1,664.350000,1
1,1,1,0,4,0.083333,1
2,1,1,1,5,0.183333,1
3,1,1,2,3,0.583333,1
4,1,1,2,5,0.075000,2
...,...,...,...,...,...,...
637778,263,263,23,2,1.965686,17
637779,263,263,23,3,38.001754,38
637780,263,263,23,4,4.094654,53
637781,263,263,23,5,3.208594,64


In [17]:


day_of_week_select = alt.binding_select(options=[0, 1, 2, 3, 4, 5, 6], labels=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], name='Day of Week: ')
day_of_week_selection = alt.selection_single(fields=['day_of_week'], bind=day_of_week_select, init={'day_of_week': 0})

pu_location_selection = alt.selection_single(fields=['PULocationID'], on='click', empty='all', name='PULocation')
do_location_selection = alt.selection_single(fields=['DOLocationID'], on='mouseover', empty='all', name='DOLocation')

trips_map = alt.Chart(alt.Data(values=tripsViz_geojson['features']))\
    .mark_geoshape(stroke='white', strokeWidth=1)\
    .encode(
        tooltip=['properties.PULocationID:O', 'properties.zone:N', 'properties.N_trips:Q'],
        color=alt.condition(
            (pu_location_selection | do_location_selection), 
            alt.Color('properties.N_trips:Q', scale=color_scale, legend=alt.Legend(title='Number of Trips (Quantiles)')), 
            alt.value('lightgray')
        ),
        opacity=alt.value(1)
    )\
    .project('albersUsa')\
    .transform_calculate(PULocationID='datum.properties.PULocationID', DOLocationID='datum.properties.PULocationID', zone='datum.properties.zone')\
    .add_selection(pu_location_selection)\
    .add_selection(do_location_selection)

bars = alt.Chart(PD)\
    .mark_circle()\
    .encode(
        x="PULocationID:O",
        y="DOLocationID:O",
        size="trip_duration:Q",
        color=alt.Color("trip_duration:Q", scale=alt.Scale(scheme="viridis")),
        opacity=alt.condition(
            (pu_location_selection & do_location_selection) |
            (pu_location_selection & ~do_location_selection),
            alt.value(1), 
            alt.value(0)
        )
    )\
    .transform_filter(pu_location_selection)\
    .transform_filter(
        do_location_selection |
        (pu_location_selection & ~do_location_selection)
    )

selected_zone_line_chart = alt.Chart(Time_byDOW)\
    .mark_line(color='orange')\
    .encode(
        x=alt.X("hour:O", axis=alt.Axis(title="Hour")),
        y=alt.Y("trip_duration:Q", axis=alt.Axis(title="Trip Duration")),
    )\
    .transform_filter(pu_location_selection)\
    .transform_filter(do_location_selection)\
    #.transform_filter(day_of_week_selection)

combined_map_bars = trips_map.properties(width=600, height=800, title='New York City Taxi Trips by Zone') | bars.properties(width=800, height=800, title='Trip Time')
chart = combined_map_bars & selected_zone_line_chart.properties(width=1400, height=400, title='Trip Duration by Hour')

#chart


In [18]:
# Time_byDOW.iloc(Time_byDOW['DOW'] == 1)

In [19]:
Time_byDOW

,PULocationID,DOLocationID,hour,DOW,trip_duration,count
0,1,1,0,1,664.350000,1
1,1,1,0,4,0.083333,1
2,1,1,1,5,0.183333,1
3,1,1,2,3,0.583333,1
4,1,1,2,5,0.075000,2
...,...,...,...,...,...,...
637778,263,263,23,2,1.965686,17
637779,263,263,23,3,38.001754,38
637780,263,263,23,4,4.094654,53
637781,263,263,23,5,3.208594,64


In [20]:


day_of_week_select = alt.binding_select(options=[0, 1, 2, 3, 4, 5, 6], labels=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], name='Day of Week: ')
day_of_week_selection = alt.selection_single(fields=['DOW'], bind=day_of_week_select, init={'DOW': 0})

pu_location_selection = alt.selection_single(fields=['PULocationID'], on='click', empty='all', name='PULocation')
do_location_selection = alt.selection_single(fields=['DOLocationID'], on='mouseover', empty='all', name='DOLocation')

trips_map = alt.Chart(alt.Data(values=tripsViz_geojson['features']))\
    .mark_geoshape(stroke='white', strokeWidth=1)\
    .encode(
        tooltip=['properties.PULocationID:O', 'properties.zone:N', 'properties.N_trips:Q'],
        color=alt.condition(
            (pu_location_selection | do_location_selection), 
            alt.Color('properties.N_trips:Q', scale=color_scale, legend=alt.Legend(title='Number of Trips (Quantiles)')), 
            alt.value('lightgray')
        ),
        opacity=alt.value(1)
    )\
    .project('albersUsa')\
    .transform_calculate(PULocationID='datum.properties.PULocationID', DOLocationID='datum.properties.PULocationID', zone='datum.properties.zone')\
    .add_selection(pu_location_selection)\
    .add_selection(do_location_selection)
    
bars = alt.Chart(PD)\
    .mark_circle()\
    .encode(
        x="PULocationID:O",
        y="DOLocationID:O",
        size="trip_duration:Q",
        color=alt.Color("trip_duration:Q", scale=alt.Scale(scheme="viridis")),
        opacity=alt.condition(
            (pu_location_selection & do_location_selection) |
            (pu_location_selection & ~do_location_selection),
            alt.value(1), 
            alt.value(0)
        )
    )\
    .transform_filter(pu_location_selection)\
    .transform_filter(
        do_location_selection |
        (pu_location_selection & ~do_location_selection)
    )

selected_zone_line_chart = alt.Chart(Time_byDOW)\
    .mark_line(color='orange')\
    .encode(
        x=alt.X("hour:O", axis=alt.Axis(title="Hour")),
        y=alt.Y("trip_duration:Q", axis=alt.Axis(title="Trip Duration")),
    )\
    .transform_filter(pu_location_selection)\
    .transform_filter(do_location_selection)\
    .transform_filter(day_of_week_selection)\
    .add_selection(day_of_week_selection)


combined_map_bars = trips_map.properties(width=600, height=800, title='New York City Taxi Trips by Zone') | bars.properties(width=800, height=800, title='Trip Time')
chart = combined_map_bars & selected_zone_line_chart.properties(width=1400, height=400, title='Trip Duration by Hour')

#chart


In [21]:
PD.head()

,PULocationID,DOLocationID,trip_duration,count
0,1,1,1,620
1,1,23,10,1
2,1,45,53,1
3,1,50,39,1
4,1,90,45,2


In [22]:


day_of_week_select = alt.binding_select(options=[0, 1, 2, 3, 4, 5, 6], labels=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], name='Day of Week: ')
day_of_week_selection = alt.selection_single(fields=['DOW'], bind=day_of_week_select, init={'DOW': 0})

pu_location_selection = alt.selection_single(fields=['PULocationID'], on='click', empty='all', name='PULocation')
do_location_selection = alt.selection_single(fields=['DOLocationID'], on='mouseover', empty='all', name='DOLocation')

trips_map = alt.Chart(alt.Data(values=tripsViz_geojson['features']))\
    .mark_geoshape(stroke='white', strokeWidth=1)\
    .encode(
        tooltip=['properties.PULocationID:O', 'properties.zone:N', 'properties.N_trips:Q'],
        color=alt.condition(
            (pu_location_selection | do_location_selection), 
            alt.Color('properties.N_trips:Q', scale=color_scale, legend=alt.Legend(title='Number of Trips (Quantiles)')), 
            alt.value('lightgray')
        ),
        opacity=alt.value(1)
    )\
    .project('albersUsa')\
    .transform_calculate(PULocationID='datum.properties.PULocationID', DOLocationID='datum.properties.PULocationID', zone='datum.properties.zone')\
    .add_selection(pu_location_selection)\
    .add_selection(do_location_selection)
    

bars = alt.Chart(PD)\
    .mark_circle()\
    .encode(
        x=alt.X("PULocationID:O", axis=alt.Axis(title="PULocationID", labelAngle=0, labelOverlap='greedy', labelSeparation=15)),
        y=alt.Y("DOLocationID:O", axis=alt.Axis(title="DOLocationID", labelAngle=0, labelOverlap='greedy', labelSeparation=15)),
        size=alt.Size("trip_duration:Q", scale=alt.Scale(range=[0, 800])),
        color=alt.Color("trip_duration:Q", scale=alt.Scale(scheme="viridis")),
        opacity=alt.condition(
            (pu_location_selection & do_location_selection) |
            (pu_location_selection & ~do_location_selection),
            alt.value(1), 
            alt.value(0)
        )
    )\
    .transform_filter(pu_location_selection)\
    .transform_filter(
        do_location_selection |
        (pu_location_selection & ~do_location_selection)
    )



step=5
x_filter = alt.Chart(alt.sequence(0, PD['PULocationID'].nunique(), step=step)).mark_rule(size=0).encode(x='value:O').add_selection(pu_location_selection)
y_filter = alt.Chart(alt.sequence(0, PD['DOLocationID'].nunique(), step=step)).mark_rule(size=0).encode(y='value:O').add_selection(do_location_selection)

bars_filtered = bars + x_filter + y_filter


selected_zone_line_chart = alt.Chart(Time_byDOW)\
    .mark_line(color='orange')\
    .encode(
        x=alt.X("hour:O", axis=alt.Axis(title="Hour")),
        y=alt.Y("trip_duration:Q", axis=alt.Axis(title="Trip Duration")),
    )\
    .transform_filter(pu_location_selection)\
    .transform_filter(do_location_selection)\
    .transform_filter(day_of_week_selection)\
    .add_selection(day_of_week_selection)


combined_map_bars = trips_map.properties(width=400, height=400, title='New York City Taxi Trips by Zone') | bars.properties(width=400, height=400, title='Trip Time')
chart = combined_map_bars & selected_zone_line_chart.properties(width=900, height=400, title='Trip Duration by Hour')

#chart


In [23]:
explanation_text = """
Click on the map to select the pickup location.
Mouse over the map to select the drop-off location.
"""

text_layer = alt.Chart({"values": [{"x": 5, "y": 5}]}).mark_text(
    align='right',
    baseline='bottom',
    fontSize=12,
    dx=0,
    dy=-20
).encode(
    x='x:Q',
    y='y:Q',
    text=alt.value(explanation_text)
)

trips_map = alt.Chart(alt.Data(values=tripsViz_geojson['features']))\
    .mark_geoshape(stroke='white', strokeWidth=1)\
    .encode(
        tooltip=['properties.PULocationID:O', 'properties.zone:N', 'properties.N_trips:Q'],
        color=alt.condition(
            (pu_location_selection | do_location_selection), 
            alt.Color('properties.N_trips:Q', scale=color_scale, legend=alt.Legend(title='Number of Trips (Quantiles)')), 
            alt.value('lightgray')
        ),
        opacity=alt.value(1)
    )\
    .project('albersUsa')\
    .transform_calculate(PULocationID='datum.properties.PULocationID', DOLocationID='datum.properties.PULocationID', zone='datum.properties.zone')\
    .add_selection(pu_location_selection)\
    .add_selection(do_location_selection)\
    .properties(width=600, height=800)

combined_map_bars = (trips_map + text_layer).properties(title='New York City Taxi Trips by Zone') &selected_zone_line_chart.properties(width=600, height=300, title='Trip Duration by Hour')
chart = combined_map_bars | bars.properties(width=700, height=1200, title='Trip Time')


chart.save("chart.html")

In [26]:
#chart